In [ ]:
import time
print('Installing dependencies')
!pip install -q accelerate==0.12.0
!pip uninstall -y diffusers
!git clone --branch updt https://github.com/TheLastBen/diffusers
!pip install -q ./diffuser

!wget -O refmdlz https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/refmdlz
!unzip -o -q refmdlz    
!wget https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/convertodiffv1.py
!wget https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/convertodiffv2.py
!wget https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertosd.py
!wget https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/smart_crop.py
print('Done, proceed')

In [10]:
CKPT_Path = "/home/zhuole/data_ssd/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt"
Model_Version = "1.5"
Session_Name = "testdb2"

IMAGES_FOLDER_OPTIONAL = "/home/zhuole/dreambooth/datasets/person"
TOKEN = 'testdb2'
Remove_existing_instance_images = True
Crop_images = False
Crop_size = "512"
Add_background = False
Upscale_image = True

Resume_Training = False
UNet_Training_Steps = 1500
UNet_Learning_Rate = 2e-5
Text_Encoder_Training_Steps = 150
Text_Encoder_Concept_Training_Steps = 0
Text_Encoder_Learning_Rate = 1e-6
Style_Training = True
Resolution = "512"

Save_Checkpoint_Every_n_Steps = True
Save_Checkpoint_Every = 500
Start_saving_from_the_step = 1000

In [ ]:
import os
import time
from IPython.display import clear_output

if CKPT_Path != "":
  clear_output()
  with capture.capture_output() as cap:
    if Model_Version=='1.5':
      !python convertodiffv1.py "$CKPT_Path" stable-diffusion-custom --v1
    elif Model_Version=='V2.1-512px':
      !python convertodiffv2.py "$CKPT_Path" stable-diffusion-custom --v2 --reference_model stabilityai/stable-diffusion-2-1-base
    elif Model_Version=='V2.1-768px':
      !python convertodiffv2.py "$CKPT_Path" stable-diffusion-custom --v2 --reference_model stabilityai/stable-diffusion-2-1
  if os.path.exists('stable-diffusion-custom/unet/diffusion_pytorch_model.bin'):
    MODEL_NAME="stable-diffusion-custom"
    print('DONE !')
  else:
    !rm -r stable-diffusion-custom
    print('Conversion error') 

    

In [3]:
import os
from IPython.display import clear_output
from IPython.utils import capture
from os import listdir
from os.path import isfile
import time

PT=""
WORKSPACE='.'
INSTANCE_NAME=Session_Name
OUTPUT_DIR=WORKSPACE+'/'+Session_Name
SESSION_DIR=WORKSPACE+'/Sessions/'+Session_Name
INSTANCE_DIR=SESSION_DIR+'/instance_images'
CONCEPT_DIR=SESSION_DIR+'/concept_images'
MDLPTH=str(SESSION_DIR+"/"+Session_Name+'.ckpt')


if os.path.exists(str(SESSION_DIR)):
  mdls = [ckpt for ckpt in listdir(SESSION_DIR) if ckpt.split(".")[-1 ]== "ckpt"]
  if not os.path.exists(MDLPTH) and '.ckpt' in str(mdls):  

    def f(n):  
      k=0
      for i in mdls:    
        if k==n:    
          !mv "$SESSION_DIR/$i" $MDLPTH
        k=k+1

    k=0
    print('No final checkpoint model found, select which intermediary checkpoint to use, enter only the number, (000 to skip):\n')

    for i in mdls:    
      print(str(k)+'- '+i)
      k=k+1
    n=input()
    while int(n)>k-1:
      n=input()  
    if n!="000":
      f(int(n))
      print('Using the model '+ mdls[int(n)]+" ...")
      time.sleep(2)
    else:
      print('Skipping the intermediary checkpoints.')
    del n

if os.path.exists(str(SESSION_DIR)) and not os.path.exists(MDLPTH):
  print('Loading session with no previous model, using the original model or the custom downloaded model')
  if MODEL_NAME=="":
    print('No model found, please specify a model.')
  else:
    print('Session Loaded, proceed to uploading instance images')

elif not os.path.exists(str(SESSION_DIR)):
    %mkdir -p "$INSTANCE_DIR"
    print('Creating session...')
    if MODEL_NAME=="":
      print('No model found, please specify a model.')
    else:
      print('Session created, proceed to uploading instance images')
      
elif os.path.exists(MDLPTH):
  print('Session found, loading the trained model ...')

Loading session with no previous model, using the original model or the custom downloaded model
Session Loaded, proceed to uploading instance images


In [4]:
import shutil
from PIL import Image
from tqdm import tqdm
from smart_crop import *

if Remove_existing_instance_images:
  if os.path.exists(str(INSTANCE_DIR)):
    !rm -r "$INSTANCE_DIR"

if not os.path.exists(str(INSTANCE_DIR)):
  %mkdir -p "$INSTANCE_DIR"

Crop_size=int(Crop_size)

if IMAGES_FOLDER_OPTIONAL!="":
  if Crop_images:
    for filename in tqdm(os.listdir(IMAGES_FOLDER_OPTIONAL), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
      extension = filename.split(".")[-1]
      identifier=filename.split(".")[0]
      new_path_with_file = os.path.join(INSTANCE_DIR, filename)
      file = Image.open(IMAGES_FOLDER_OPTIONAL+"/"+filename)
      width, height = file.size
      if file.size !=(Crop_size, Crop_size):  
        image = crop_image(file, Crop_size)    
        # side_length = min(width, height)
        # left = (width - side_length)/2
        # top = (height - side_length)/2
        # right = (width + side_length)/2
        # bottom = (height + side_length)/2
        # image = file.crop((left, top, right, bottom))
        # image = image.resize((Crop_size, Crop_size))
        if (extension.upper() == "JPG"):
            image[0].save(new_path_with_file, format="JPEG", quality = 100)
        else:
            image[0].save(new_path_with_file, format=extension.upper())
      else:
        !cp "$IMAGES_FOLDER_OPTIONAL/$filename" "$INSTANCE_DIR"

  else:
    for filename in tqdm(os.listdir(IMAGES_FOLDER_OPTIONAL), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
      %cp -r "$IMAGES_FOLDER_OPTIONAL/$filename" "$INSTANCE_DIR"
 
  print('Done, proceed to the next cell')


  |███████████████| 19/19 Uploaded

Done, proceed to the next cell


In [5]:
from PIL import Image
import glob
import os
import shutil

files = glob.glob(os.path.join(INSTANCE_DIR, '*'))
os.makedirs(INSTANCE_DIR+"_tmp", exist_ok=True)
for idx, i in enumerate(files):
  if Add_background:
    im = Image.open(i).convert("RGBA")
    new_image = Image.new("RGBA", im.size, "WHITE")
    new_image.paste(im, (0, 0), im)
    new_image.convert('RGB').save(os.path.join(INSTANCE_DIR+"_tmp", '%s(%d).jpg'%(TOKEN, idx+1)), quality=100)
  else:
    im = Image.open(i)
    im.convert('RGB').save(os.path.join(INSTANCE_DIR+"_tmp", '%s(%d).jpg'%(TOKEN, idx+1)), quality=100)

shutil.rmtree(INSTANCE_DIR)
os.rename(INSTANCE_DIR+'_tmp', INSTANCE_DIR)

if Upscale_image:
  INSTANCE_DIR_NEW = INSTANCE_DIR+'_upscaled'
  with capture.capture_output() as cap:
    !python ./Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus_anime_6B -i "$INSTANCE_DIR" -o "$INSTANCE_DIR_NEW"
  files = glob.glob(os.path.join(INSTANCE_DIR_NEW, '*'))
  os.makedirs(INSTANCE_DIR+"_tmp", exist_ok=True)
  for idx, i in enumerate(files):
    im = Image.open(i)
    im.convert('RGB').save(os.path.join(INSTANCE_DIR+"_tmp", '%s(%d).jpg'%(TOKEN, idx+1)), quality=100)
  shutil.rmtree(INSTANCE_DIR_NEW)
  shutil.rmtree(INSTANCE_DIR)
  os.rename(INSTANCE_DIR+'_tmp', INSTANCE_DIR)

In [15]:

import os
from subprocess import getoutput
from IPython.display import clear_output
import time
import random

if os.path.exists(INSTANCE_DIR+"/.ipynb_checkpoints"):
  %rm -r $INSTANCE_DIR"/.ipynb_checkpoints"

if os.path.exists(CONCEPT_DIR+"/.ipynb_checkpoints"):
  %rm -r $CONCEPT_DIR"/.ipynb_checkpoints"  

try:
   resume
   if resume and not Resume_Training:
     print('Overwrite your previously trained model ?, answering "yes" will train a new model, answering "no" will resume the training of the previous model?  yes or no ?')
     while True:
        ansres=input('')
        if ansres=='no':
          Resume_Training = True
          del ansres
          break
        elif ansres=='yes':
          Resume_Training = False
          resume= False
          break
except:
  pass

while not Resume_Training and MODEL_NAME=="":
  print('No model found, use the "Model Download" cell to download a model.')
  time.sleep(5)


MODELT_NAME=MODEL_NAME
untlr=UNet_Learning_Rate
txlr=Text_Encoder_Learning_Rate

trnonltxt=""
if UNet_Training_Steps==0:
   trnonltxt="--train_only_text_encoder"

Seed='' 

Style=""
if Style_Training:
  Style="--Style"

Res=int(Resolution)

fp16 = True

if Seed =='' or Seed=='0':
  Seed=random.randint(1, 999999)
else:
  Seed=int(Seed)

GC="--gradient_checkpointing"

if fp16:
  prec="fp16"
else:
  prec="no"

s = getoutput('nvidia-smi')
if 'A100' in s:
  GC=""

precision=prec

resuming=""
if Resume_Training and os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
  MODELT_NAME=OUTPUT_DIR
  print('Resuming Training...m')
  resuming="Yes"
elif Resume_Training and not os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
  print('Previous model not found, training a new model...')
  MODELT_NAME=MODEL_NAME
  while MODEL_NAME=="":
    print('No model found, use the "Model Download" cell to download a model.')
    time.sleep(5)

V2=False
if os.path.getsize(MODELT_NAME+"/text_encoder/pytorch_model.bin") > 670901463:
  V2=True

Enable_text_encoder_training= True 
Enable_Text_Encoder_Concept_Training= True

if Text_Encoder_Training_Steps==0:
   Enable_text_encoder_training= False
else:
  stptxt=Text_Encoder_Training_Steps

if Text_Encoder_Concept_Training_Steps==0:
   Enable_Text_Encoder_Concept_Training= False
else:
  stptxtc=Text_Encoder_Concept_Training_Steps

if Enable_text_encoder_training:
  Textenc="--train_text_encoder"
else:
  Textenc=""

if Save_Checkpoint_Every==None:
  Save_Checkpoint_Every=1

stp=0

if Start_saving_from_the_step==None:
  Start_saving_from_the_step=0
if (Start_saving_from_the_step < 200):
  Start_saving_from_the_step=Save_Checkpoint_Every
stpsv=Start_saving_from_the_step
if Save_Checkpoint_Every_n_Steps:
  stp=Save_Checkpoint_Every

def dump_only_textenc(trnonltxt, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, precision, Training_Steps):
    !accelerate launch ./diffusers/examples/dreambooth/train_dreambooth.py \
    $trnonltxt \
    --image_captions_filename \
    --train_text_encoder \
    --dump_only_text_encoder \
    --pretrained_model_name_or_path="$MODELT_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --instance_prompt="$PT" \
    --seed=$Seed \
    --resolution=512 \
    --mixed_precision=$precision \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 $GC \
    --use_8bit_adam \
    --learning_rate=$txlr \
    --lr_scheduler="polynomial" \
    --lr_warmup_steps=0 \
    --max_train_steps=$Training_Steps

def train_only_unet(stpsv, stp, SESSION_DIR, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, Res, precision, Training_Steps):
    clear_output()
    if resuming=="Yes":
      print('Resuming Training...')    
    print('Training the UNet...')
    !accelerate launch ./diffusers/examples/dreambooth/train_dreambooth.py \
    $Style \
    --image_captions_filename \
    --train_only_unet \
    --save_starting_step=$stpsv \
    --save_n_steps=$stp \
    --Session_dir=$SESSION_DIR \
    --pretrained_model_name_or_path="$MODELT_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --instance_prompt="$PT" \
    --seed=$Seed \
    --resolution=$Res \
    --mixed_precision=$precision \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 $GC \
    --use_8bit_adam \
    --learning_rate=$untlr \
    --lr_scheduler="polynomial" \
    --lr_warmup_steps=0 \
    --max_train_steps=$Training_Steps

if Enable_text_encoder_training :
  print('Training the text encoder...')
  if os.path.exists(OUTPUT_DIR+'/'+'text_encoder_trained'):
    %rm -r $OUTPUT_DIR"/text_encoder_trained"
  dump_only_textenc(trnonltxt, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, precision, Training_Steps=stptxt)

if Enable_Text_Encoder_Concept_Training:
  if os.path.exists(CONCEPT_DIR):
    if os.listdir(CONCEPT_DIR)!=[]:
      clear_output()
      if resuming=="Yes":
        print('Resuming Training...')    
      print('Training the text encoder on the concept...')
      dump_only_textenc(trnonltxt, MODELT_NAME, CONCEPT_DIR, OUTPUT_DIR, PT, Seed, precision, Training_Steps=stptxtc)
    else:
      clear_output()
      if resuming=="Yes":
        print('Resuming Training...')      
      print('No concept images found, skipping concept training...')
      time.sleep(8)
  else:
      clear_output()
      if resuming=="Yes":
        print('Resuming Training...')
      print('No concept images found, skipping concept training...')
      time.sleep(8)

if UNet_Training_Steps!=0:
  train_only_unet(stpsv, stp, SESSION_DIR, MODELT_NAME, INSTANCE_DIR, OUTPUT_DIR, PT, Seed, Res, precision, Training_Steps=UNet_Training_Steps)
    

if os.path.exists(INSTANCE_NAME+'/unet/diffusion_pytorch_model.bin'):
  prc="--fp16" if precision=="fp16" else ""
  if V2:
    !python ./diffusers/scripts/convertosdv2.py $prc $OUTPUT_DIR $SESSION_DIR/$Session_Name".ckpt"
    clear_output()
    if os.path.exists(SESSION_DIR+"/"+INSTANCE_NAME+'.ckpt'):
      clear_output()
      print("DONE, the CKPT model is in your drive in the sessions folder")     
    else:
      print("Something went wrong")     
  else:  
    clear_output()
    if precision=="no":
      !sed -i '226s@.*@@' ./convertosd.py
    !sed -i '201s@.*@    model_path = "{OUTPUT_DIR}"@' ./convertosd.py
    !sed -i '202s@.*@    checkpoint_path= "{SESSION_DIR}/{Session_Name}.ckpt"@' ./convertosd.py
    !python convertosd.py
    clear_output()
    if os.path.exists(SESSION_DIR+"/"+INSTANCE_NAME+'.ckpt'):      
      print("DONE, the CKPT model is in your drive in the sessions folder")
    else:
      print("Something went wrong")
    
else:
  print("Something went wrong")

Training the UNet...


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--num_cpu_threads_per_process` was set to `14` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/opt/anaconda/anaconda3/envs/ldm/lib/python3.8/site-packages/diffusers/utils/deprecation_utils.py:35: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddpm.DDPMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  warnings.warn(warning + message, FutureWarning)
Progress:|                         |:   1%| | 16/1500 [00:06<09:00,  2.75it/s, l  

In [16]:
FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a scheduler, please use <class 'diffusers.schedulers.scheduling_ddpm.DDPMScheduler'>.from_pretrained(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0. warnings.warn(warning + message, FutureWarning

SyntaxError: invalid syntax (476867567.py, line 1)